In [1]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars
from examples.CoDec_Research.code.analysis.evaluate_construal_actions import evaluate_construals, get_best_construals_likelihood



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-06 21:48:19.743435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746582499.773748  259472 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746582499.782964  259472 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746582499.826803  259472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746582499.826858  259472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746582499.826859  259472 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [2]:

### Parameters for Inference ###
heuristic_params = {"ego_distance": 0.5} # Parameters for weighing the heuristics

### Specify Environment Configuration ###

# |Location to store (and find pre-computed) simulation results
simulation_results_path = "examples/CoDec_Research/results/simulation_results/"
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]

# |Location to store simulation results
out_dir = "examples/CoDec_Research/results/simulation_results/"

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set scenario path
dataset_path = 'data/processed/construal'

# |Set simulator config
max_agents = training_config.max_controlled_agents   # Get total vehicle count
num_parallel_envs = 2
total_envs = 4
device = "cpu" # cpu just because we're in a notebook
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# |Set construal config
construal_size = 1
observed_agents_count = max_agents - 1      # Agents observed except self (used for vector sizes)
sample_size = 1                             # Number of samples to calculate expected utility of a construal

# |Other changes to variables
training_config.max_controlled_agents = 1    # Control only the first vehicle in the environment
total_envs = min(total_envs, len(listdir(dataset_path)))


### Instantiate Variables ###

env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config = training_config,
                                                                                device = device,
                                                                                num_parallel_envs = num_parallel_envs,
                                                                                dataset_path = dataset_path,
                                                                                max_agents = max_agents,
                                                                                total_envs = total_envs,
                                                                                sim_agent_path= "daphne-cornelisse/policy_S10_000_02_27",
                                                                            )


In [3]:

### Select Construals for Baseline Data ###
scene_constr_dict = None

#2# |Generate Construal Execution Values through simulator sampling
default_values = None

#2# |Check if saved data is available
for srFile in simulation_results_files:
    if "construal_vals" in srFile:
        with open(srFile, 'rb') as opn_file:
            default_values = pickle.load(opn_file)
    else:
        continue

if default_values is None:
    default_values, traj_obs, ground_truth = generate_all_construal_trajnval(out_dir=out_dir,
                                                                                sim_agent=sim_agent,
                                                                                observed_agents_count=observed_agents_count,
                                                                                construal_size=construal_size,
                                                                                num_parallel_envs=num_parallel_envs,
                                                                                max_agents=max_agents,
                                                                                sample_size=sample_size,
                                                                                device=device,
                                                                                train_loader=train_loader,
                                                                                env=env,
                                                                                env_multi_agent=env_multi_agent,
                                                                                generate_animations=False)

#2# |Generate Construal Heuristic Values (Heuristic 1: Distance from ego)
heuristic_values = get_constral_heurisrtic_values(env, train_loader, default_values, heuristic_params=heuristic_params)

#2# |Sample from construal values
def sample_construals(heuristic_values: dict, sample_count: int) -> dict:
    """
    Sample construals based on heuristic values.
    """
    sampled_construals = {}
    for scene_name, construal_info in heuristic_values.items():
        constr_indices, constr_values = zip(*construal_info.items())
        constr_values_softmax = torch.nn.functional.softmax(torch.tensor(constr_values), dim=0)
        sampled_indices = torch.multinomial(constr_values_softmax, num_samples=sample_count, \
                                                            replacement=False).tolist()
        sampled_construals[scene_name] = {constr_indices[i]: constr_values[i] for i in sampled_indices}

    return sampled_construals

scene_constr_dict = sample_construals(heuristic_values, sample_count=2)




Processing Waymo batches: 100%|██████████| 2/2 [00:00<00:00,  8.82it/s]


In [4]:
heuristic_values

{'tfrecord-00101-of-01000_71': {(0,): np.float64(1.0),
  (1,): np.float64(0.9548024362736818),
  (5,): np.float64(0.629016666487872),
  (9,): np.float64(0.5742991115584066),
  (10,): np.float64(0.9042478755727865),
  (42,): np.float64(0.613054930528357),
  (43,): np.float64(0.5385952434420198),
  (0, 1, 5, 9, 10, 42, 43): np.float64(0.7448594662661605)},
 'tfrecord-00117-of-01000_240': {(0,): np.float64(1.0),
  (1,): np.float64(0.7215077800397827),
  (2,): np.float64(0.5381755943648112),
  (3,): np.float64(0.6078264890552836),
  (4,): np.float64(0.9401724415680937),
  (5,): np.float64(0.700456969135634),
  (7,): np.float64(0.9385660133622136),
  (8,): np.float64(0.6320838417348973),
  (9,): np.float64(0.744795752946394),
  (10,): np.float64(0.6767799318279193),
  (11,): np.float64(0.7732606874164518),
  (12,): np.float64(0.827249699708235),
  (13,): np.float64(0.7074301356948892),
  (14,): np.float64(-0.20800941945850976),
  (15,): np.float64(-0.19140329129862552),
  (16,): np.float64(

In [5]:
### Generate Synthetic Ground Truth for Selected Construals (Baseline Data on Which to Perform Inference) ###
state_action_pairs = None

# |Check if saved data is available
for srFile in simulation_results_files:
    if "baseline_state_action_pairs" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        state_action_pairs = pickle.load(opn_file)

if state_action_pairs is None:
    state_action_pairs = generate_baseline_data(out_dir=out_dir,
                                                sim_agent=sim_agent,
                                                num_parallel_envs=num_parallel_envs,
                                                max_agents=max_agents,
                                                sample_size=1,
                                                device=device,
                                                train_loader=train_loader,
                                                env=env,
                                                env_multi_agent=env_multi_agent,
                                                observed_agents_count=observed_agents_count,
                                                construal_size=construal_size,
                                                selected_construals=scene_constr_dict,
                                                generate_animations=False)


In [6]:
### Compute Construal Log Likelihoods ###
construal_action_likelihoods = None

# |Check if saved data is available
for srFile in simulation_results_files:
    if "log_likelihood_measures" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        construal_action_likelihoods = pickle.load(opn_file)

if construal_action_likelihoods is None:
    construal_action_likelihoods = evaluate_construals(state_action_pairs, construal_size, sim_agent, out_dir, device=device)

# |Clear memory for large variable, once it has served its purpose
# del state_action_pairs




In [7]:

### Sanity Check ###
scene_constr_dict = None
scene_constr_diff_dict = None

# |Check if saved data is available
for srFile in simulation_results_files:
    if "highest_construal_dict_log_likelihood_diff" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_diff_dict = pickle.load(opn_file)
    if "highest_construal_dict_log_likelihood" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_dict = pickle.load(opn_file)

if scene_constr_dict is None:
    scene_constr_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir)
if scene_constr_diff_dict is None:
    scene_constr_diff_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir, likelihood_key="log_likelihood_diff")


In [8]:
for scene_name_, scene_info_ in construal_action_likelihoods.items():
    print(f"Scene: {scene_name_}")
    for base_construal_name_, base_construal_info_ in scene_info_.items():
        print_dict = {}
        for test_construal_name_, test_construal_info_ in base_construal_info_.items():
            for sample_num_, sample_info_ in test_construal_info_.items():
                # print_dict.update({(base_construal_name_, test_construal_name_, sample_num_): abs(sample_info_['log_likelihood_diff'])})
                print_dict.update({(base_construal_name_, test_construal_name_, sample_num_): sample_info_['log_likelihood']})
        print(dict(sorted(print_dict.items(), key=lambda item: item[1])))

Scene: tfrecord-00101-of-01000_71
{((10,), (0, 1, 5, 9, 10, 42, 43), 0): tensor(45.0291, requires_grad=True), ((10,), (1,), 0): tensor(46.5550, requires_grad=True), ((10,), (5,), 0): tensor(53.8457, requires_grad=True), ((10,), (10,), 0): tensor(58.1080, requires_grad=True), ((10,), (0,), 0): tensor(58.2338, requires_grad=True), ((10,), (42,), 0): tensor(58.2338, requires_grad=True), ((10,), (43,), 0): tensor(58.2338, requires_grad=True), ((10,), (9,), 0): tensor(58.5760, requires_grad=True)}
{((0, 1, 5, 9, 10, 42, 43), (0, 1, 5, 9, 10, 42, 43), 0): tensor(40.8187, requires_grad=True), ((0, 1, 5, 9, 10, 42, 43), (1,), 0): tensor(42.0083, requires_grad=True), ((0, 1, 5, 9, 10, 42, 43), (5,), 0): tensor(57.5896, requires_grad=True), ((0, 1, 5, 9, 10, 42, 43), (10,), 0): tensor(61.6216, requires_grad=True), ((0, 1, 5, 9, 10, 42, 43), (0,), 0): tensor(61.6605, requires_grad=True), ((0, 1, 5, 9, 10, 42, 43), (42,), 0): tensor(61.6605, requires_grad=True), ((0, 1, 5, 9, 10, 42, 43), (43,), 0

In [ ]:
### Inference Logic ###

# |Get probability of lambda values
get_constral_heurisrtic_values_partial = partial(get_constral_heurisrtic_values, env=env, 
                                                 train_loader=train_loader, default_values=default_values)
p_lambda = {}
for curr_lambda in np.linspace(0,1,11):
    curr_lambda = curr_lambda.item()
    heuristic_params = {"ego_distance": curr_lambda}
    curr_heuristic_values = get_constral_heurisrtic_values_partial(heuristic_params=heuristic_params)
    p_lambda[curr_lambda] = {}
    for scene_name, scene_info in curr_heuristic_values.items():
        p_lambda[curr_lambda][scene_name] = {}
        sampled_construals = construal_action_likelihoods[scene_name]
        for base_construal, construal_info in sampled_construals.items():
            curr_p_lambda = []
            for construal_indx, construal_heur_value in scene_info.items():
                sample_num = 0
                p_a = -1*construal_info[construal_indx][sample_num]['log_likelihood'].item()
                curr_p_lambda.append(( torch.exp(torch.tensor(p_a))*construal_heur_value.item() ).item())
            p_lambda[curr_lambda][scene_name][base_construal] = sum(curr_p_lambda)/len(curr_p_lambda)

Processing Waymo batches: 100%|██████████| 2/2 [00:00<00:00,  8.80it/s]


In [10]:
p_lambda

{0.0: {'tfrecord-00101-of-01000_71': {(10,): 4.231859807463398e-21,
   (0, 1, 5, 9, 10, 42, 43): 3.0545672030782197e-19},
  'tfrecord-00117-of-01000_240': {(9,): 1.6047202913959698e-42,
   (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17): 0.0},
  'tfrecord-00143-of-01000_179': {(3,): 9.744119957836842e-42,
   (14,): 8.747277200461541e-37},
  'tfrecord-00144-of-01000_130': {(0,): 2.8752489111303885e-21,
   (0, 7, 59): 2.4832118498845487e-21}},
 0.1: {'tfrecord-00101-of-01000_71': {(10,): 4.04763485194575e-21,
   (0, 1, 5, 9, 10, 42, 43): 2.9286250032032814e-19},
  'tfrecord-00117-of-01000_240': {(9,): 1.5152707394232354e-42,
   (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17): 0.0},
  'tfrecord-00143-of-01000_179': {(3,): 9.419655667960904e-42,
   (14,): 8.459647309532444e-37},
  'tfrecord-00144-of-01000_130': {(0,): 2.8334202495177537e-21,
   (0, 7, 59): 2.4465675064103795e-21}},
 0.2: {'tfrecord-00101-of-01000_71': {(10,): 3.863410300373034e-21,
   (0, 1, 5, 9, 1

In [11]:
# |Get product over lambda probability across sampled construals
lamda_inference = {}
for curr_lambda, scene_info in p_lambda.items():
    lamda_inference[curr_lambda] = np.prod([val for scene_name, construal_info in scene_info.items() for val in construal_info.values() if val > 0])
lamda_inference

{0.0: np.float64(1.2623652522307264e-199),
 0.1: np.float64(9.922264755770672e-200),
 0.2: np.float64(7.719908795380786e-200),
 0.30000000000000004: np.float64(5.939352956207508e-200),
 0.4: np.float64(4.513566144459845e-200),
 0.5: np.float64(3.383149847462046e-200),
 0.6000000000000001: np.float64(2.4966225046927738e-200),
 0.7000000000000001: np.float64(1.81068286216386e-200),
 0.8: np.float64(1.287282791478903e-200),
 0.9: np.float64(8.943332017481195e-201),
 1.0: np.float64(6.0481356264385754e-201)}